# Часть 1. Объединение и категоризация датасетов по городам.

**Цель** - объединить множество похожих датасетов `moscow1`, `moscow2`, `moscow3` и т.д. в один для дальнейшей удобной работы, а также разбить все датасеты по городам.

**Ход работы**:
1. Импортировать нужные инструменты, посмотреть обзорно на датасеты.
2. Объединить уже готовые датасеты в один.
3. Создать новые датасеты с городами, которые нам не даны в отдельном датасете.

#### 1. Импортируем нужные инструменты.

In [115]:
import pandas as pd

pd.set_option('display.max_columns', None)
# displaying all the columns

#### 2. Загрузим датасеты.

In [116]:
ekaterinburg = pd.read_csv('ekaterinburg.csv')
kazan = pd.read_csv('kazan.csv')
krasnodar = pd.read_csv('krasnodar.csv')

nizhny_novgorod = pd.read_csv('nizhny_novgorod.csv')
novosibirsk = pd.read_csv('novosibirsk.csv')
rostov_na_donu = pd.read_csv('rostov_na_donu.csv')

samara = pd.read_csv('samara.csv')
ufa = pd.read_csv('ufa.csv')
voronezh = pd.read_csv('voronezh.csv')

In [117]:
moscow1 = pd.read_csv('moscow1.csv')
moscow2 = pd.read_csv('moscow2.csv')
moscow3 = pd.read_csv('moscow3.csv')

moscow4 = pd.read_csv('moscow4.csv')
moscow5 = pd.read_csv('moscow5.csv')
moscow6 = pd.read_csv('moscow6.csv')

In [118]:
petersburg1 = pd.read_csv('petersburg1.csv')
petersburg2 = pd.read_csv('petersburg2.csv')

In [119]:
othercities1 = pd.read_csv('othercities1.csv')
othercities2 = pd.read_csv('othercities2.csv')
othercities3 = pd.read_csv('othercities3.csv')

othercities4 = pd.read_csv('othercities4.csv')
othercities5 = pd.read_csv('othercities5.csv')

othercities6 = pd.read_csv('othercities6.csv')
othercities7 = pd.read_csv('othercities7.csv')

#### 3. Объединим датасеты `moscow` в один, `other_cities` в один и `petersburg` в один.

In [138]:
moscow = pd.concat([moscow1, moscow2, moscow3,
                    moscow4, moscow5, moscow6], ignore_index = True)

petersburg = pd.concat([petersburg1,
                        petersburg2], ignore_index = True)

othercities = pd.concat([othercities1, othercities2, othercities3, othercities4,
                         othercities5, othercities6, othercities6], ignore_index = True)

#### 4. Проверим, есть ли в `othercities` информация о заказах из городов, которые уже разделены на датасеты.

In [139]:
cities = ['Москва', 'Санкт-Петербург', 'Екатеринбург',
          'Казань', 'Краснодар', 'Нижний Новгород',
          'Новосибирск', 'Ростов-на-Дону', 'Самара',
          'Уфа', 'Воронеж']

cities_duplicates = []

for othercity in othercities['address_city'].value_counts().index:
    for city in cities:
        if city in othercity:
            cities_duplicates.append(othercity)

In [140]:
print(f"Населенные пункты в cities_duplicates, которые уже есть в наших датасетах: {len(cities_duplicates)} штук")

Населенные пункты в cities_duplicates, которые уже есть в наших датасетах: 49 штук


#### 5. Как видим, довольно много повторяющихся строк, поэтому вынесем все строки в отдельные датасеты, а потом склеим с уже существующими.

Извлечем повторяющиеся строки в отдельный датасет с дубликатами.

In [141]:
main_dup = othercities.query('address_city in @cities_duplicates')

In [142]:
main_dup['address_city'].value_counts().head(10)

Москва                                      66809
Москва, поселение Десеновское                 734
Краснодарский край, городской округ Сочи      646
Москва, поселение Десёновское                 243
Москва, поселение Рязановское                 212
Москва, поселение Сосенское                   191
Москва, поселение Щаповское                   152
Москва, поселение Внуковское                  147
Москва, поселение Первомайское                137
Москва, поселение Краснопахорское             105
Name: address_city, dtype: int64

В принципе, существенную часть датасета составляет только информация с заказами из Москвы, поэтому оставим только их и присоединим к уже существующему датасету `moscow`.

In [143]:
moscow.shape

(2865497, 19)

In [144]:
main_dup = main_dup.query('address_city == "Москва"')
moscow = pd.concat([moscow, main_dup], ignore_index = True)

In [145]:
moscow.shape

(2932306, 19)

#### 6. Удалим значения c Москвой из `other_cities` и посмотрим на крупные города в `other_cities`.

In [155]:
othercities = othercities[othercities['address_city'] != 'Москва']
othercities.shape

(2782379, 19)

In [174]:
cnt = 0
for index, value in othercities['address_city'].value_counts().items():
    print(index, value)
    cnt +=1
    if cnt == 11: break

Тюмень 97361
Пермь 89403
Челябинск 87385
Калининград 80020
Волгоград 74401
Сочи 69046
Омск 67431
Саратов 61274
Балашиха 54440
Красноярск 53808
Хабаровск 52669


Возьмем ТОП10 городов и создадим для них отдельные датасеты. Балашиху не берем, её добавим в датасет с Москвой.

In [183]:
tumen = othercities.query('address_city == "Тюмень"').reset_index()
perm = othercities.query('address_city == "Пермь"').reset_index()
chelyabinsk = othercities.query('address_city == "Челябинск"').reset_index()

kaliningrad = othercities.query('address_city == "Калининград"').reset_index()
volgograd = othercities.query('address_city == "Волгоград"').reset_index()
sochi = othercities.query('address_city == "Сочи"').reset_index()

omsk = othercities.query('address_city == "Омск"').reset_index()
saratov = othercities.query('address_city == "Саратов"').reset_index()
krasnoyarsk = othercities.query('address_city == "Красноярск"').reset_index()
habarovsk = othercities.query('address_city == "Хабаровск"').reset_index()

In [185]:
bala = othercities.query('address_city == "Балашиха"')

In [187]:
moscow.shape

(2932306, 19)

In [188]:
moscow = pd.concat([moscow, bala])

In [191]:
moscow.shape

(2986746, 19)

Удалим ТОП11 городов из `othercities`.

In [193]:
to_delete = ['Тюмень', 'Пермь', 'Челябинск', 'Калининград', 'Волгоград',
             'Сочи', 'Омск', 'Саратов', 'Красноярск', 'Хабаровск', 'Балашиха']

othercities = othercities.query('address_city not in @to_delete')

#### 7. Объединим новые и преобразованные датасеты в `.csv` файлы для дальнейшего использования.

In [195]:
moscow.to_csv('moscow.csv')
petersburg.to_csv('petersburg.csv')

othercities.to_csv('othercities.csv')

In [196]:
tumen.to_csv('tumen.csv')
perm.to_csv('perm.csv')
chelyabinsk.to_csv('chelyabinsk.csv')

kaliningrad.to_csv('kaliningrad.csv')
volgograd.to_csv('volgograd.csv')
sochi.to_csv('sochi.csv')

omsk.to_csv('omsk.csv')
saratov.to_csv('saratov.csv')
krasnoyarsk.to_csv('krasnoyarsk.csv')
habarovsk.to_csv('habarovsk.csv')


## Вывод:
*Что было сделано:*

1. Загрузили датасеты и сразу объединили их в один
2. Проверили недочеты в othercities и добавили уже имеющиеся города в датасеты этих городов.
3. Очистили добавленную информацию в другие датасеты из `othercities`.
4. Взяли еще ТОП10 городов из `othercities` и раскидали их по датасетам.
5. Объединили и провели финальную стадию, а именно эскпорт новых датасетов в формат `.csv` для дальнейшей работы